# Deploy Keras Models on Google Cloud Platform (GCP) and make predictions over the web

<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/140XQWEHsaOluFh5tyN13CI9uV0eMCkpi">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/bankarpranit26/deep-learning/blob/master/learnings/Deploy_Keras_on_GCP.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    View source on GitHub</a>
  </td>
</table>

## Import the required libraries

In [0]:
try:
    %tensorflow_version 1.x
except:
    pass

In [0]:
# TensorFlow
import tensorflow as tf

# Helper libraries
import numpy as np
import os

from google.colab import auth as google_auth
google_auth.authenticate_user()

# tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

## Upload, extract and load your model

In [0]:
from google.colab import files
import shutil

# You can either upload your zip file in this way or go to 'Files' and upload there
zip_name = files.upload()

# This is because my zip file contains the model folder inside it.
# Based on your zip file structure, adjust the path accordingly.
extract_path = '/content/'
shutil.unpack_archive(list(zip_name.keys())[0], extract_path , 'zip')

In [0]:
model_path = '/content/linear_model/1'            # Your model version path
model = tf.keras.models.load_model(model_path)

In [0]:
round(model.predict([0]).tolist()[0][0])

## Deploy your model to GCP

In [0]:
BUCKET_NAME='bucket-2711298'
PROJECT_NAME = 'project-2711298'
MODEL_NAME = 'model'
MODEL_VERSION = 'v1'
TF_VERSION = '1.15'
PYTHON_VERSION = '3.7'
REGION = 'us-central1'

In [0]:
GS_PATH = 'gs://' + BUCKET_NAME + '/' + MODEL_NAME
export_path = tf.saved_model.save(model, GS_PATH)

In [0]:
! gcloud config set project $PROJECT_NAME

! gcloud ai-platform models create model --regions $REGION

! gcloud ai-platform versions create $MODEL_VERSION \
--model $MODEL_NAME \
--runtime-version $TF_VERSION \
--python-version $PYTHON_VERSION \
--framework tensorflow \
--origin $GS_PATH

## Make predictions on your model

In [0]:
with open('/content/test_data.json', 'w') as f:
  f.write('[0]')

! gcloud ai-platform predict \
--model $MODEL_NAME \
--version $MODEL_VERSION \
--json-instances '/content/test_data.json'